# Tiền xử lý dữ liệu (Data Preprocessing)

## Tải lên và kiểm tra các loại dữ liệu. Phân loại các dữ liệu dạng Object

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import joblib
from pathlib import Path

# import các thư viện cần thiết cho ColumnTransformer và Pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.exceptions import NotFittedError

In [50]:
# Thiết lập đường dẫn
current_dir = Path.cwd()
data_dir = current_dir.parent / 'data'
preprocessing_dir = current_dir

train_path = data_dir / 'train_new.csv'
test_path = data_dir / 'test_new.csv'

In [51]:
# Tải dữ liệu train
df_train_clean = pd.read_csv(train_path)
df_test_clean = pd.read_csv(test_path)
print("Đã tải dữ liệu thành công")

Đã tải dữ liệu thành công


In [52]:
# dữ liệu dạng object
df_train_clean.dtypes[df_train_clean.dtypes=='object']

MSZoning         object
Street           object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
LandSlope        object
Neighborhood     object
Condition1       object
Condition2       object
BldgType         object
HouseStyle       object
RoofStyle        object
RoofMatl         object
Exterior1st      object
Exterior2nd      object
MasVnrType       object
ExterQual        object
ExterCond        object
Foundation       object
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
Heating          object
HeatingQC        object
CentralAir       object
Electrical       object
KitchenQual      object
Functional       object
FireplaceQu      object
GarageType       object
GarageFinish     object
GarageQual       object
PavedDrive       object
SaleType         object
SaleCondition    object
dtype: object

In [53]:
# các dữ liệu khác
df_train_clean.dtypes[df_train_clean.dtypes!='object']

Id                   int64
MSSubClass           int64
LotFrontage        float64
LotArea              int64
OverallQual          int64
OverallCond          int64
MasVnrArea         float64
BsmtUnfSF            int64
LowQualFinSF         int64
BedroomAbvGr         int64
KitchenAbvGr         int64
TotRmsAbvGrd         int64
Fireplaces           int64
GarageCars           int64
WoodDeckSF           int64
PoolArea             int64
MiscVal              int64
MoSold               int64
SalePrice          float64
houseage             int64
houseremodelage      int64
totalsf              int64
totalarea            int64
totalbath          float64
totalporchsf         int64
dtype: object

#### Để chuẩn bị cho feature encoding, ở đây chúng tôi sẽ thực hiện bước tách các feature với target ra:

In [54]:
num_cols = df_train_clean.select_dtypes(include=['int64', 'float64']).columns
num_cols = num_cols.drop('SalePrice')  # Loại bỏ target

#### Sau khi xác định được tất cả các cột dạng object chúng tôi sẽ phân loại chúng thành 2 nhóm như sau:
* Ordinal Encoding: các cột có thứ tự (cấp độ, chất lượng, đánh giá, ...), các biến có thứ tự sẵn, có mức độ quan trọng
* One-Hot Encoding: các cột không có thứ tự (tên khu, kiểu nhà, loại đường, ...), categories độc lập
#### Dựa vào các dữ liệu các biến object phía trên cũng như thông tin mà Kaggle cung cấp chúng tôi sẽ chia nhanh các cột dữ liệu vào hai danh sách sau

In [55]:
# các cột Ordinal Encoding
ode_cols = ['LotShape', 'LandContour', 'Utilities', 'LandSlope', 
            'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
            'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 
            'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 
            'PavedDrive', 'ExterQual', 'ExterCond']

In [56]:
# các cột One-Hot Encoding
ohe_cols = ['MSZoning', 'Street', 'LotConfig', 'Neighborhood', 
            'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
            'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
            'MasVnrType', 'Foundation', 'Heating', 'GarageType',
            'SaleType', 'SaleCondition']

## Biến đổi các cột để xử lý tất cả các loại feature cùng lúc.

In [57]:
# Tạo Numerical Pipeline
num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [58]:
# Tạo Ordinal Encoding Pipeline
ode_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ode', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

In [59]:
# Tạo One-Hot Encoding Pipeline
ohe_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [60]:
# Biến đổi các cột
col_trans = ColumnTransformer(transformers=[
    ('num_p', num_pipeline, num_cols),
    ('ode_p', ode_pipeline, ode_cols),
    ('ohe_p', ohe_pipeline, ohe_cols),
    ],
    remainder='passthrough',
    n_jobs=-1)

In [61]:
# Tạo một Pipeline chính bao bọc các cột đã biến đổi:
pipeline = Pipeline(steps=[
    ('preprocessing', col_trans)
])

## Tách các biến đặc trưng và các biến mục tiêu trước khi tiền xử lý dữ liệu và huấn luyện mô hình

In [76]:
# Tách features (đặc trưng) và target (mục tiêu)
features = df_train_clean.drop('SalePrice', axis=1)
target = df_train_clean['SalePrice']

In [ ]:
# Áp dụng pipeline tiền xử lý
features_preprocessed = pipeline.fit_transform(features)
print(f"Kích thước features sau tiền xử lý: {features_preprocessed.shape}")

Kích thước features sau tiền xử lý: (1439, 192)


#### Lấy ra tên cột sau khi transform để sử dụng sau này

In [64]:
# lấy tên cột sau khi transform
def get_feature_names_from_pipeline(pipeline):
    
    # Lấy ColumnTransformer từ pipeline
    col_trans = pipeline.named_steps['preprocessing']
    feature_names = []
    
    # Lấy tên cột numerical
    num_features = col_trans.transformers_[0][2]  # numerical columns
    num_processor = col_trans.transformers_[0][1]
    num_scaled_features = num_features.tolist()
    feature_names.extend(num_scaled_features)
    
    # Lấy tên cột ordinal
    ode_features = col_trans.transformers_[1][2]  # ordinal columns
    # Ordinal encoding giữ nguyên số lượng cột
    feature_names.extend(ode_features)
    
    # Lấy tên cột one-hot
    ohe_features = col_trans.transformers_[2][2]  # one-hot columns
    ohe_processor = col_trans.transformers_[2][1]
    ohe_encoder = ohe_processor.named_steps['ohe']
    
    # Lấy tên categories từ one-hot encoder
    ohe_feature_names = []
    for i, feature in enumerate(ohe_features):
        categories = ohe_encoder.categories_[i]
        for category in categories:
            ohe_feature_names.append(f"{feature}_{category}")
    
    feature_names.extend(ohe_feature_names)
    return feature_names

# Lấy tên cột sau khi transform
feature_names = get_feature_names_from_pipeline(pipeline)
print(f"Số lượng đặc trưng: {len(feature_names)}")

Số lượng đặc trưng: 192


In [ ]:
# Chia dữ liệu thành tập train và test
features_train, features_test, target_train, target_test = train_test_split(
    features_preprocessed, 
    target, 
    test_size=0.2, 
    random_state=25
# chia tập train - test theo mức 80 - 20
)

print(f"Kích thước tập train - Features: {features_train.shape}, Target: {target_train.shape}")
print(f"Kích thước tập test - Features: {features_test.shape}, Target: {target_test.shape}")

Kích thước tập train - Features: (1151, 192), Target: (1151,)
Kích thước tập test - Features: (288, 192), Target: (288,)


In [66]:
# Transform tập test với pipeline đã được fit từ tập train
test_features_preprocessed = pipeline.transform(df_test_clean)

print(f"Kích thước tập test sau xử lý: {test_features_preprocessed.shape}")

# So sánh với tập train
print(f"Tập train: {features_preprocessed.shape}")
print(f"Tập test:  {test_features_preprocessed.shape}")

if features_preprocessed.shape[1] == test_features_preprocessed.shape[1]:
    print("Đã kiểm tra số lượng đặc trưng nhất quán giữa tập train và tập test")

Kích thước tập test sau xử lý: (1459, 192)
Tập train: (1439, 192)
Tập test:  (1459, 192)
Đã kiểm tra số lượng đặc trưng nhất quán giữa tập train và tập test


In [67]:
# Tạo DataFrame với tên cột
train_processed_df = pd.DataFrame(features_preprocessed, columns=feature_names)
test_processed_df = pd.DataFrame(test_features_preprocessed, columns=feature_names)

# Lưu với tên cột
train_processed_df.to_csv('../data/data_afterProcessed/train_processed.csv', index=False)
test_processed_df.to_csv('../data/data_afterProcessed/test_processed.csv', index=False)

print("Đã lưu tập train và tập test đã xử lý tên cột")

Đã lưu tập train và tập test đã xử lý tên cột


In [68]:
# Lưu target và pipeline
target.to_csv('../data/data_afterProcessed/target.csv', index=False)
joblib.dump(pipeline, '../preprocessing/preprocessing_pipeline.pkl')

['../preprocessing/preprocessing_pipeline.pkl']

## Xử lý tương tự cho các tập dữ liệu chưa qua FE để huấn luyện thêm các mô hình cho thực nghiệm dữ liệu thô

In [69]:
# Đường dẫn đến file dữ liệu
train_mv_path = "../data/data_afterMVAnalysis/train_clean_rmMV.csv"
test_mv_path = "../data/data_afterMVAnalysis/test_clean_rmMV.csv"   

# Tải dữ liệu và tách target
train_mv = pd.read_csv(train_mv_path)
test_mv = pd.read_csv(test_mv_path)

target_mv = train_mv['SalePrice']
features_mv = train_mv.drop('SalePrice', axis=1)

print(f"Train features: {features_mv.shape}")
print(f"Test features: {test_mv.shape}")
print(f"Target: {target_mv.shape}")

Train features: (1439, 73)
Test features: (1459, 73)
Target: (1439,)


In [70]:
# Lưu target cho Exp1
target_mv_path = "../data/data_afterMVAnalysis/target.csv"
target_mv.to_csv(target_mv_path, index=False)

print(f"Đã lưu target vào: {target_mv_path}")
print(f"Khoảng giá nhà (Target range): ${target_mv.min():,.0f} - ${target_mv.max():,.0f}")

Đã lưu target vào: ../data/data_afterMVAnalysis/target.csv
Khoảng giá nhà (Target range): $35,311 - $625,000


In [71]:
# Tách numerical và categorical columns
numerical_cols = features_mv.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = features_mv.select_dtypes(include=['object']).columns

print(f"Numerical columns: {len(numerical_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

preprocessor_simple = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numerical_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_cols)
    ]
)

# Fit và transform
features_mv_processed = preprocessor_simple.fit_transform(features_mv)
test_mv_processed = preprocessor_simple.transform(test_mv)

print(f"cấu trúc đặc trưng và mục tiêu sau tiền xử lý dữ liệu: {features_mv_processed.shape}")

Numerical columns: 36
Categorical columns: 37
cấu trúc đặc trưng và mục tiêu sau tiền xử lý dữ liệu: (1439, 268)


In [72]:
# lấy ra tên cột sau khi transform
def get_feature_names_simple(column_transformer):
    feature_names = []
    
    # Numerical features (giữ nguyên tên)
    num_features = column_transformer.transformers_[0][2]
    feature_names.extend(num_features)
    
    # Categorical features (one-hot encoded)
    cat_processor = column_transformer.transformers_[1][1]
    cat_encoder = cat_processor.named_steps['ohe']
    cat_features = column_transformer.transformers_[1][2]
    
    for i, feature in enumerate(cat_features):
        categories = cat_encoder.categories_[i]
        for category in categories:
            feature_names.append(f"{feature}_{category}")
    
    return feature_names

# Lấy tên cột
feature_names_simple = get_feature_names_simple(preprocessor_simple)
print(f"Số lượng đặc trưng còn lại sau tiền xử lý dữ liệu: {len(feature_names_simple)}")

Số lượng đặc trưng còn lại sau tiền xử lý dữ liệu: 268


In [73]:
# Tạo DataFrame với tên cột
features_mv_processed_df = pd.DataFrame(features_mv_processed, columns=feature_names_simple)
test_mv_processed_df = pd.DataFrame(test_mv_processed, columns=feature_names_simple)

In [74]:
features_mv_processed_path = "../data/data_afterMVAnalysis/train_features_rmMV.csv"
test_mv_processed_path = "../data/data_afterMVAnalysis/test_features_rmMV.csv"

features_mv_processed_df.to_csv(features_mv_processed_path, index=False)
test_mv_processed_df.to_csv(test_mv_processed_path, index=False)

print("Đã lưu các đặc trưng đã qua tiền xử lý dữ liệu")

Đã lưu các đặc trưng đã qua tiền xử lý dữ liệu


# Kết thúc

In [75]:
!jupyter nbconvert --to html "preprocessing.ipynb"

[NbConvertApp] Converting notebook preprocessing.ipynb to html
[NbConvertApp] Writing 339056 bytes to preprocessing.html
